In [1]:
import os
import random
import time

from reNN.base import *

from sklearn.linear_model import LogisticRegression

from tqdm import tqdm

In [10]:
ndims = 500
set_size = 10000

#embedding_dir = "/Users/rob/ML_Data/x86_vectors/lstm/"+str(ndims)+"/"
#label_dir = embedding_dir
#embedding_dir = "/Users/rob/ML_Data/x86_vectors/lstm/"+str(ndims)+"-raw/"
embedding_dir = "/Users/rob/ML_Data/x86_vectors/byte_3grams/500/"
label_dir = "/Users/rob/ML_Data/x86_vectors/byte_3grams/labels/"
reNN_dir = "/Users/rob/ML_Data/code_standard/"

In [11]:
all_reNN_files = []

for root, dir, files in os.walk(reNN_dir):
    for f in files:
        all_reNN_files.append(root+"/"+f)
        
print len(all_reNN_files)



6354


In [12]:
loader = reNN_binary()


collections = []
class1_funcs = []
class2_funcs = []
class3_funcs = []
class4_funcs = []

#class1_types = ["reactos", "windows10"]
#class2_types = ["Arch_Linux"]

class1_types = ["visual studio"]
class2_types = ["gcc"]

for fname in tqdm(all_reNN_files):
    try:
        loader = loader.load(fname)
    except:
        continue
    for func in loader.functions:
        collections.append(loader.cc_flags)
        if loader.functions[func][1] > 20:
            if loader.cc_name == "visual studio":
                if loader.cc_flags == "debug":
                    class1_funcs.append(loader.md5+":"+loader.name+":"+func)
                else:
                    class2_funcs.append(loader.md5+":"+loader.name+":"+func)
            elif loader.cc_name == "gcc":
                if loader.cc_flags == "-O1":
                    class3_funcs.append(loader.md5+":"+loader.name+":"+func)
                else:
                    class4_funcs.append(loader.md5+":"+loader.name+":"+func)
                    
print len(class1_funcs)
print len(class2_funcs)
print len(class3_funcs)
print len(class4_funcs)
print set(collections)
            

133481
211611
549625
197986
set([u'-O3', u'debug', u'-O1', u'release'])


In [13]:
def load_labels(label_file):
    labels = []
    with open(label_file, "r") as lfile:
        for line in lfile:
            labels.append(''.join([i + ":" for i in line.split(":")[1:]])[:-1].strip())
    return labels

In [14]:
def load_data_set(func_names, ndims):
    functions_by_hash = {}
    func_names.sort()
    counter = 0
    data_matrix = np.zeros((len(func_names), ndims))
    for f in func_names:
        file_hash = f.split(":")[0]
        function_name = ''.join([i + ":" for i in f.split(":")[2:]])[:-1]
        try:
            functions_by_hash[file_hash].append(function_name)
        except:
            functions_by_hash[file_hash] = [function_name]
    for file_hash in functions_by_hash:
        try:
            buff = np.load(embedding_dir+file_hash+".npy")
            buff_labels = load_labels(label_dir+file_hash+".labels")
        except:
            continue
        indexes = []
        for func_name in functions_by_hash[file_hash]:
            try:
                indexes.append(buff_labels.index(func_name))
            except:
                continue
        for i in indexes:
            data_matrix[counter] = buff[i]
            counter = counter + 1
    buff = data_matrix[:counter]
    return buff
        
        

In [15]:
def test_data_set():
    hard_matches = []
    soft_matches = []
    for i in range(0,1):
        print "Starting loop %d..." % i
        start = time.time()
        class1_sample = random.sample(class1_funcs, 100000)
        class2_sample = random.sample(class2_funcs, 100000)
        class3_sample = random.sample(class3_funcs, 100000)
        class4_sample = random.sample(class4_funcs, 100000)

        class1_train_names = class1_sample[:80000]
        class2_train_names = class2_sample[:80000]
        class3_train_names = class3_sample[:80000]
        class4_train_names = class4_sample[:80000]
        class1_test_names = class1_sample[80000:]
        class2_test_names = class2_sample[80000:]
        class3_test_names = class3_sample[80000:]
        class4_test_names = class4_sample[80000:]

        class1_train = load_data_set(class1_train_names, ndims)
        class2_train = load_data_set(class2_train_names, ndims)
        class3_train = load_data_set(class3_train_names, ndims)
        class4_train = load_data_set(class4_train_names, ndims)
        train_classes = np.zeros(class1_train.shape[0]+class2_train.shape[0]+
                                 class3_train.shape[0]+class4_train.shape[0], dtype = np.int16)
        c2_offset = class1_train.shape[0]
        c3_offset = c2_offset + class2_train.shape[0]
        c4_offset = c3_offset + class3_train.shape[0]
        train_classes[c2_offset:c3_offset] = 1
        train_classes[c3_offset:c4_offset] = 2
        train_classes[c4_offset:] = 3
        train_data = np.concatenate((class1_train, class2_train, class3_train, class4_train))
        print class1_train.shape
        print class2_train.shape
        print class3_train.shape
        print class4_train.shape
        print train_classes

        class1_test = load_data_set(class1_test_names, ndims)
        class2_test = load_data_set(class2_test_names, ndims)
        class3_test = load_data_set(class3_test_names, ndims)
        class4_test = load_data_set(class4_test_names, ndims)
        test_classes = np.zeros(class1_test.shape[0]+class2_test.shape[0]+
                                class3_test.shape[0]+class4_test.shape[0], dtype = np.int16)
        c2_offset = class1_test.shape[0]
        c3_offset = c2_offset + class2_test.shape[0]
        c4_offset = c3_offset + class3_test.shape[0]
        test_classes[c2_offset:c3_offset] = 1
        test_classes[c3_offset:c4_offset] = 2
        test_classes[c4_offset:] = 3
        test_data = np.concatenate((class1_test, class2_test, class3_test, class4_test))
        print class1_test.shape
        print class2_test.shape
        print test_classes

        classifier = LogisticRegression(multi_class="multinomial", solver="sag")
        classifier.fit(train_data, train_classes)


        probs = classifier.predict_proba(test_data)
        hard = 0
        soft = 0

        for p in xrange(0,test_data.shape[0]):
            pred = np.argmax(probs[p])
            if pred == test_classes[p]:
                hard = hard + 1
            if pred == 0 or pred == 1 and test_classes[p] == 0 or test_classes[p] == 1:
                soft = soft + 1
            if pred == 2 or pred == 3 and test_classes[p] == 2 or test_classes[p] == 3:
                soft = soft + 1

        print float(soft)/test_data.shape[0]
        print float(hard)/test_data.shape[0]
        soft_matches.append(float(soft)/test_data.shape[0])
        hard_matches.append(float(hard)/test_data.shape[0])
        print "Loop completed in %.3f seconds" % (time.time() - start)
    return soft_matches, hard_matches

In [16]:
scores = test_data_set()

Starting loop 0...


IndexError: index 2049 is out of bounds for axis 0 with size 2049

In [ ]:
print scores

In [ ]:
probs = classifier.predict_proba(test_data)

In [ ]:
print probs

In [ ]:
hard = 0
soft = 0

for p in xrange(0,test_data.shape[0]):
    pred = np.argmax(probs[p])
    if pred == test_classes[p]:
        hard = hard + 1
    if pred == 0 or pred == 1 and test_classes[p] == 0 or test_classes[p] == 1:
        soft = soft + 1
    if pred == 2 or pred == 3 and test_classes[p] == 2 or test_classes[p] == 3:
        soft = soft + 1
        
print float(soft)/test_data.shape[0]
print float(hard)/test_data.shape[0]

In [ ]:
print hard

In [ ]:
print pred

In [ ]:
print test_classes